In [86]:
import requests
from lxml import html
import pandas as pd
from bs4 import BeautifulSoup
import re
import sys
from datetime import datetime
import time
import numpy as np
import json
from config import *

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select

import time
import json
import re
from tqdm import tqdm

In [ ]:
#strava

# stats= https://www.strava.com/activities/586018101/streams?stream_types%5B%5D=timer_time&stream_types%5B%5D=altitude&stream_types%5B%5D=heartrate&stream_types%5B%5D=cadence&stream_types%5B%5D=temp&stream_types%5B%5D=distance&stream_types%5B%5D=grade_smooth&stream_types%5B%5D=time&stream_types%5B%5D=grade_adjusted_distance&_=1537842990620
# latlng = https://www.strava.com/activities/586018101/streams?stream_types%5B%5D=timer_time&stream_types%5B%5D=latlng&_=1537842990619

pete = "https://www.strava.com/athletes/16102568/training/log?feature=sidebar-progress-goals-v2"
adam= "https://www.strava.com/pros/11620352#interval?interval=201607&interval_type=week&chart_type=miles&year_offset=2"
jermey = "https://www.strava.com/athletes/1150260#graph_date_range?chart_type=miles&interval_type=week&interval=201640&year_offset=2"
jason = "https://www.strava.com/athletes/13823509#interval?interval=201621&interval_type=week&chart_type=miles&year_offset=2"

In [88]:
#pull date, distance, time, pace

activities = []
html_text = []

        #load webpage
browser = webdriver.Chrome() 
browser.get("https://www.strava.com/login") #navigate to the page
browser.implicitly_wait(5)

            # login = browser.find_element_by_xpath('//*[@id="global-nav"]/div/ul/li[5]/a').click()

#sign-in
username = browser.find_element_by_id('email')
username.clear()
username.send_keys(username_c)

password = browser.find_element_by_id('password')
password.clear()
password.send_keys(strava_pw)

browser.find_element_by_id("login-button").click()

#search
browser.find_element_by_xpath('//*[@id="global-search-field"]').send_keys('pete kostelnick')
browser.implicitly_wait(2)
browser.find_element_by_xpath('//*[@id="global-search-button"]').click()
browser.implicitly_wait(2)
browser.find_element_by_xpath('/html/body/div[1]/div[3]/div/ul/li[1]/div[1]/div[2]/h4/a').click()


#change year of data
browser.get('https://www.strava.com/pros/16102568#interval_type?chart_type=miles&interval_type=month&interval=201610&year_offset=2')
time.sleep(2)
# browser.find_element_by_xpath('//*[@id="interval-graph-controls"]/ul/li[2]/ul/li[2]/a').click()

for x in range(10,11):
    time.sleep(2)
    #click month bar in graph
    browser.find_element_by_xpath(f'//*[@id="interval-2016{str(x)}"]/div/a').click()

    #activity links
    time.sleep(4)

    feed = browser.find_element_by_class_name('feed')
    webdriver.ActionChains(browser).move_to_element(feed)

    html = feed.get_attribute('innerHTML')
    
    html_text.append(html)

#for september
time.sleep(2)
#click month bar in graph
browser.find_element_by_xpath(f'//*[@id="interval-201609"]/div/a').click()

#activity links
time.sleep(4)

feed = browser.find_element_by_class_name('feed')
webdriver.ActionChains(browser).move_to_element(feed)

html = feed.get_attribute('innerHTML')

html_text.append(html)

In [89]:
#pull name, date, activity stats for each entry on page
#skip if all values aren't present

activity_details = []

for html in html_text:
    bs = BeautifulSoup(html, 'html.parser')
    container = bs.find_all('div', attrs={"class":"entry-container"})
    for a in container:
        try:
            name = a.find_all('a', attrs={'class':"entry-athlete"})[0].get_text().strip('\n').split('\n')[0]
            date = a.find_all('time', attrs={'class':"timestamp"})[0].get_text().strip('\n')
            dist = a.find_all('li', attrs={'title':"Distance"})[0].get_text()
            pace = a.find_all('li', attrs={'title':"Pace"})[0].get_text()
            dur = a.find_all('li', attrs={'title':"Time"})[0].get_text()
            activity_details.append([name, date, dist, pace, dur])
        except:
            pass
activity_details

[['Pete Kostelnick',
  'October 29, 2016 at 4:35 PM',
  '3.19 mi',
  '8:46 /mi',
  '28m 0s'],
 ['Pete Kostelnick',
  'October 24, 2016 at 12:05 PM',
  '30.14 mi',
  '10:24 /mi',
  '5h 13m'],
 ['Pete Kostelnick',
  'October 24, 2016 at 12:00 AM',
  '30.30 mi',
  '11:43 /mi',
  '5h 55m'],
 ['Pete Kostelnick',
  'October 23, 2016 at 10:58 AM',
  '31.56 mi',
  '10:43 /mi',
  '5h 38m'],
 ['Pete Kostelnick',
  'October 22, 2016 at 11:01 AM',
  '32.55 mi',
  '11:01 /mi',
  '5h 58m'],
 ['Pete Kostelnick',
  'October 22, 2016 at 3:33 AM',
  '41.15 mi',
  '9:46 /mi',
  '6h 41m'],
 ['Pete Kostelnick',
  'October 21, 2016 at 11:54 AM',
  '30.90 mi',
  '10:55 /mi',
  '5h 37m'],
 ['Pete Kostelnick',
  'October 21, 2016 at 3:52 AM',
  '41.26 mi',
  '10:47 /mi',
  '7h 24m'],
 ['Pete Kostelnick',
  'October 20, 2016 at 11:34 AM',
  '34.23 mi',
  '10:49 /mi',
  '6h 10m'],
 ['Pete Kostelnick',
  'October 19, 2016 at 11:22 AM',
  '32.23 mi',
  '10:54 /mi',
  '5h 51m'],
 ['Pete Kostelnick',
  'October 19, 

In [90]:
#convert to dataframe and name columns
pete_df = pd.DataFrame(activity_details, columns=['name', 'date', 'distance', 'pace', 'duration'])
pete_df

,name,date,distance,pace,duration
0,Pete Kostelnick,"October 29, 2016 at 4:35 PM",3.19 mi,8:46 /mi,28m 0s
1,Pete Kostelnick,"October 24, 2016 at 12:05 PM",30.14 mi,10:24 /mi,5h 13m
2,Pete Kostelnick,"October 24, 2016 at 12:00 AM",30.30 mi,11:43 /mi,5h 55m
3,Pete Kostelnick,"October 23, 2016 at 10:58 AM",31.56 mi,10:43 /mi,5h 38m
4,Pete Kostelnick,"October 22, 2016 at 11:01 AM",32.55 mi,11:01 /mi,5h 58m
5,Pete Kostelnick,"October 22, 2016 at 3:33 AM",41.15 mi,9:46 /mi,6h 41m
6,Pete Kostelnick,"October 21, 2016 at 11:54 AM",30.90 mi,10:55 /mi,5h 37m
7,Pete Kostelnick,"October 21, 2016 at 3:52 AM",41.26 mi,10:47 /mi,7h 24m
8,Pete Kostelnick,"October 20, 2016 at 11:34 AM",34.23 mi,10:49 /mi,6h 10m
9,Pete Kostelnick,"October 19, 2016 at 11:22 AM",32.23 mi,10:54 /mi,5h 51m


In [91]:
#convert date to datetime object
pete_df['date'] = pete_df['date'].str.replace('at', '')
pete_df.date=pd.to_datetime(pete_df.date)
pete_df

,name,date,distance,pace,duration
0,Pete Kostelnick,2016-10-29 16:35:00,3.19 mi,8:46 /mi,28m 0s
1,Pete Kostelnick,2016-10-24 12:05:00,30.14 mi,10:24 /mi,5h 13m
2,Pete Kostelnick,2016-10-24 00:00:00,30.30 mi,11:43 /mi,5h 55m
3,Pete Kostelnick,2016-10-23 10:58:00,31.56 mi,10:43 /mi,5h 38m
4,Pete Kostelnick,2016-10-22 11:01:00,32.55 mi,11:01 /mi,5h 58m
5,Pete Kostelnick,2016-10-22 03:33:00,41.15 mi,9:46 /mi,6h 41m
6,Pete Kostelnick,2016-10-21 11:54:00,30.90 mi,10:55 /mi,5h 37m
7,Pete Kostelnick,2016-10-21 03:52:00,41.26 mi,10:47 /mi,7h 24m
8,Pete Kostelnick,2016-10-20 11:34:00,34.23 mi,10:49 /mi,6h 10m
9,Pete Kostelnick,2016-10-19 11:22:00,32.23 mi,10:54 /mi,5h 51m


In [93]:
pete_df['duration_sum'] =pete_df['duration'].str.split(' ')
pete_df

,name,date,distance,pace,duration,duration_sum
0,Pete Kostelnick,2016-10-29 16:35:00,3.19 mi,8:46 /mi,28m 0s,"[28m, 0s]"
1,Pete Kostelnick,2016-10-24 12:05:00,30.14 mi,10:24 /mi,5h 13m,"[5h, 13m]"
2,Pete Kostelnick,2016-10-24 00:00:00,30.30 mi,11:43 /mi,5h 55m,"[5h, 55m]"
3,Pete Kostelnick,2016-10-23 10:58:00,31.56 mi,10:43 /mi,5h 38m,"[5h, 38m]"
4,Pete Kostelnick,2016-10-22 11:01:00,32.55 mi,11:01 /mi,5h 58m,"[5h, 58m]"
5,Pete Kostelnick,2016-10-22 03:33:00,41.15 mi,9:46 /mi,6h 41m,"[6h, 41m]"
6,Pete Kostelnick,2016-10-21 11:54:00,30.90 mi,10:55 /mi,5h 37m,"[5h, 37m]"
7,Pete Kostelnick,2016-10-21 03:52:00,41.26 mi,10:47 /mi,7h 24m,"[7h, 24m]"
8,Pete Kostelnick,2016-10-20 11:34:00,34.23 mi,10:49 /mi,6h 10m,"[6h, 10m]"
9,Pete Kostelnick,2016-10-19 11:22:00,32.23 mi,10:54 /mi,5h 51m,"[5h, 51m]"


In [94]:
#create column for duration in minutes
def total_duration(x):
    min_sum = 0
    for item in x:
        min_sum += convert_to_minutes(item)
    return min_sum

def convert_to_minutes(time):
    time_type = time[-1];
    if time_type == "h":
        return int(time[0:-1]) * 60
    if time_type =="m":
        return int(time[0:-1])
    else:
        return 0

pete_df['duration_sum'] = pete_df['duration_sum'].map(total_duration)
pete_df

,name,date,distance,pace,duration,duration_sum
0,Pete Kostelnick,2016-10-29 16:35:00,3.19 mi,8:46 /mi,28m 0s,28
1,Pete Kostelnick,2016-10-24 12:05:00,30.14 mi,10:24 /mi,5h 13m,313
2,Pete Kostelnick,2016-10-24 00:00:00,30.30 mi,11:43 /mi,5h 55m,355
3,Pete Kostelnick,2016-10-23 10:58:00,31.56 mi,10:43 /mi,5h 38m,338
4,Pete Kostelnick,2016-10-22 11:01:00,32.55 mi,11:01 /mi,5h 58m,358
5,Pete Kostelnick,2016-10-22 03:33:00,41.15 mi,9:46 /mi,6h 41m,401
6,Pete Kostelnick,2016-10-21 11:54:00,30.90 mi,10:55 /mi,5h 37m,337
7,Pete Kostelnick,2016-10-21 03:52:00,41.26 mi,10:47 /mi,7h 24m,444
8,Pete Kostelnick,2016-10-20 11:34:00,34.23 mi,10:49 /mi,6h 10m,370
9,Pete Kostelnick,2016-10-19 11:22:00,32.23 mi,10:54 /mi,5h 51m,351


In [218]:
#remove /mi from pace and convert to decimal 
pete_df['pace'] =pete_df['pace'].str.replace('/mi', '')

def express_pace_decimal(row):
    try:
        minutes_part = float(row.split(":")[0])
        seconds_part = float(row.split(":")[1])
        minute_fraction = seconds_part / 60
        return (minutes_part +  minute_fraction)
    except:
        return '0'
pete_df['pace_fraction'] = pete_df['pace'].map(express_pace_decimal) 
pete_df

C:\Users\snowr\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\snowr\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


name                     object
date             datetime64[ns]
distance                 object
pace                     object
duration                 object
duration_sum              int64
pace_fraction           float64
dtype: object

In [206]:
#filter the data for run dates
pete_summary = pete_df.sort_values(by=['date'])
pete_summary =pete_df[(pete_df['date'] > '2016-09-03 19:00:00') & (pete_df['date'] < '2016-10-29 16:35:00')]
pete_summary

,name,date,distance,pace,duration,duration_sum,pace_fraction
1,Pete Kostelnick,2016-10-24 12:05:00,30.14 mi,10:24,5h 13m,313,10.400000
2,Pete Kostelnick,2016-10-24 00:00:00,30.30 mi,11:43,5h 55m,355,11.716667
3,Pete Kostelnick,2016-10-23 10:58:00,31.56 mi,10:43,5h 38m,338,10.716667
4,Pete Kostelnick,2016-10-22 11:01:00,32.55 mi,11:01,5h 58m,358,11.016667
5,Pete Kostelnick,2016-10-22 03:33:00,41.15 mi,9:46,6h 41m,401,9.766667
6,Pete Kostelnick,2016-10-21 11:54:00,30.90 mi,10:55,5h 37m,337,10.916667
7,Pete Kostelnick,2016-10-21 03:52:00,41.26 mi,10:47,7h 24m,444,10.783333
8,Pete Kostelnick,2016-10-20 11:34:00,34.23 mi,10:49,6h 10m,370,10.816667
9,Pete Kostelnick,2016-10-19 11:22:00,32.23 mi,10:54,5h 51m,351,10.900000
10,Pete Kostelnick,2016-10-19 03:57:00,41.33 mi,9:45,6h 42m,402,9.750000


In [207]:
pete_summary['distance'] = pete_summary['distance'].str.replace('mi', '')
pete_summary


,name,date,distance,pace,duration,duration_sum,pace_fraction
1,Pete Kostelnick,2016-10-24 12:05:00,30.14,10:24,5h 13m,313,10.400000
2,Pete Kostelnick,2016-10-24 00:00:00,30.30,11:43,5h 55m,355,11.716667
3,Pete Kostelnick,2016-10-23 10:58:00,31.56,10:43,5h 38m,338,10.716667
4,Pete Kostelnick,2016-10-22 11:01:00,32.55,11:01,5h 58m,358,11.016667
5,Pete Kostelnick,2016-10-22 03:33:00,41.15,9:46,6h 41m,401,9.766667
6,Pete Kostelnick,2016-10-21 11:54:00,30.90,10:55,5h 37m,337,10.916667
7,Pete Kostelnick,2016-10-21 03:52:00,41.26,10:47,7h 24m,444,10.783333
8,Pete Kostelnick,2016-10-20 11:34:00,34.23,10:49,6h 10m,370,10.816667
9,Pete Kostelnick,2016-10-19 11:22:00,32.23,10:54,5h 51m,351,10.900000
10,Pete Kostelnick,2016-10-19 03:57:00,41.33,9:45,6h 42m,402,9.750000


In [213]:
#split dattime into 2 columns so I can grouby day
pete_summary['day'] = [d.date() for d in pete_summary['date']]
pete_summary['time'] = [d.time() for d in pete_summary['date']]

#convert to distance to number
pete_summary['distance']= pd.to_numeric(pete_summary['distance'], errors='coerce')
new = pete_summary.groupby(['day', 'name']).agg(['sum', 'mean'])[['distance', 'pace_fraction', 'duration_sum']].reset_index()
new.columns = new.columns.map('_'.join)
new.drop(['distance_mean', 'pace_fraction_sum', 'duration_sum_mean'], axis =1, inplace=True)
# new = new['day_', 'name_', 'distance_sum', 'pace_fraction_mean', 'duration_sum_sum']
new.columns = ['day', 'name', 'total_distance', 'avg_pace', 'duration']
pete_run_summary = new
pete_run_summary


,day,name,total_distance,avg_pace,duration
0,2016-09-12,Pete Kostelnick,80.90,9.294444,741
1,2016-09-13,Pete Kostelnick,55.14,9.725000,536
2,2016-09-14,Pete Kostelnick,15.66,13.483333,211
3,2016-09-15,Pete Kostelnick,72.95,10.700000,746
4,2016-09-16,Pete Kostelnick,73.63,10.183333,750
5,2016-09-17,Pete Kostelnick,69.73,11.400000,752
6,2016-09-19,Pete Kostelnick,70.37,11.066667,777
7,2016-09-20,Pete Kostelnick,70.63,10.316667,727
8,2016-09-21,Pete Kostelnick,70.51,10.333333,730
9,2016-09-22,Pete Kostelnick,72.35,10.366667,746


In [214]:
pete_run_summary.to_csv('pete_summary.csv')